In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1 = pd.read_csv('../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')

In [ ]:
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.describe()

Age lies between 18-70.

Annual income lies between 15k−137k.

spending score lies between 1 and 99


## **EDA**

## Gender

In [ ]:
sns.countplot(df1['Gender'])

Costumers are more of Female gender

## Age

In [ ]:
df1['Age'].value_counts().head()

In [ ]:
ageplot = sns.distplot(df1['Age'],bins=5)

In [ ]:
sns.boxplot(x='Gender',y='Age',data=df1)

we observe that median age of Male customers is greater than Female customers and more distributed as well compared to Female customers.

## Annual Income

In [ ]:
sns.distplot(df1['Annual Income (k$)'],bins=7)

In [ ]:
sns.boxplot(x='Gender',y='Annual Income (k$)',data=df1)

* we observe that customers with income range around 50-75k $ are more in number
* Mean Annual income of Male customers is more but Female customers Annual Income is more distributed compared to Male.



## Spending Score

In [ ]:
sns.distplot(df1['Spending Score (1-100)'],bins=20)

In [ ]:
sns.boxplot(x='Gender',y='Spending Score (1-100)',data=df1)

Observations

*     customers with spending in between 40-60 are more in number.
*     Medain Spending is almost same for both the genders and Spending score is more distributed incase of Male Gender.



In [ ]:
sns.displot(data=df1,x='Annual Income (k$)',y='Spending Score (1-100)')
#plt.plot(x='Annual Income (k$)',y='Spending Score (1-100)',data=df1)
sns.jointplot(x='Annual Income (k$)',y='Spending Score (1-100)',data=df1,hue='Gender')

Insights from the above plot. It looks like we can form 5 clusters as follows

*     low income, low spending
*     low income high spending
*     High income low spending
*     High income , high spending
*     Medium income, and Medium Spending (this is where the most of the distribution lies from the above plot)

Looks like Gender is of no importance as its not showing any particular pattern in the above plot.


In [ ]:
sns.displot(data=df1,x='Age',y='Spending Score (1-100)')
sns.jointplot(x='Age',y='Spending Score (1-100)',data=df1,hue='Gender')


*     Spending Score is less when the age>60
*     No clear relation with gender



In [ ]:
sns.displot(data=df1,x='Age',y='Annual Income (k$)')
sns.jointplot(x='Age',y='Annual Income (k$)',data=df1,hue='Gender')



*     No Clear insights from the above distribution.
*     Every point of Age covered almost all points of Annual Income
*     Seems to be a uniform distribution



In [ ]:
sns.heatmap(df1.drop(['CustomerID'],axis=1).corr(),annot=True)


Inference from EDA:

*     There appaers to have slight negative correlation between Age and Spending
*     Age neither followed any pattern nor formed amny clusters w.r.t Age,Spending or income.
*     There appears to be clear cluster formation incase of Spendinhg and Income



In [ ]:
df2 = df1.drop(columns=['CustomerID'])

In [ ]:
from sklearn.cluster import KMeans,DBSCAN
from sklearn.metrics import silhouette_score,silhouette_samples
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
df2.head()

## **Preprocessing Data**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

### Scaling the data

In [ ]:
scaler = MinMaxScaler()  
def Normalize(df):
  for each in df.columns.tolist():
    if str(df[each].dtypes) == 'int64':
      df[each] = scaler.fit_transform(df[each].values.reshape(-1,1))

In [ ]:
Normalize(df2)

In [ ]:
df2['Gender'] = df2['Gender'].apply(lambda x: 1 if x == "Male" else 0)

In [ ]:
df2.head()

## **AnnualIncome Vs SpendingScore**

## With K-Means

Since AnuualIncome vs Spending Score has shown some pattern. Let's See the segmentation using income vs spending score

In [ ]:
X = df2[['Annual Income (k$)','Spending Score (1-100)']]

In [ ]:
inertia = []
for n in range(3,11):
  clstr = KMeans(n_clusters=n,init='k-means++',random_state=42)
  clstr.fit(X)
  inertia.append(clstr.inertia_)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(np.arange(3,11),inertia,'-')
plt.plot(np.arange(3,11),inertia,'o',c='r')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Plot')
plt.show()

From the above plot it is clear that number of clusters 5 would be valid as discussed.

In [ ]:
clstr1 = KMeans(n_clusters=5,init='k-means++',random_state=42)
clstr1.fit(X)
labels1 = clstr1.labels_
centroids1 = clstr1.cluster_centers_
X['Clusters'] = labels1

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(X['Annual Income (k$)'],X['Spending Score (1-100)'],c=labels1,s=50,cmap='viridis')
plt.scatter(centroids1[:,0],centroids1[:,1],marker='^',c='black',s=200,alpha=0.5)
plt.xlabel('Annual Income')
plt.ylabel('Spending Score')
plt.title('AnnualIncome vs SpendingScore')
plt.grid(False)
plt.show()

### Validation using Silhouette score

In [ ]:
silhouette_score(X[['Annual Income (k$)','Spending Score (1-100)']],X['Clusters'])

In [ ]:
silh_score = []
for k in range(2,7):
  fig,ax = plt.subplots(1,2,figsize=(15,5))
  km = KMeans(n_clusters=k,init='k-means++',random_state=42)
  clstr2 = km.fit(X[['Annual Income (k$)','Spending Score (1-100)']])
  labels2 = clstr2.labels_
  centroids2 = clstr2.cluster_centers_
  silhoute_vals = silhouette_samples(X[['Annual Income (k$)','Spending Score (1-100)']],labels2)
  silh_score.append(silhouette_score(X[['Annual Income (k$)','Spending Score (1-100)']],labels2,metric='euclidean'))

  y_ticks = []
  y_lower = y_upper = 0
  for i,cluster in enumerate(np.unique(labels2)):
    cluster_silhouette_vals = silhoute_vals[labels2 == cluster]
    cluster_silhouette_vals.sort()
    y_upper +=len(cluster_silhouette_vals)

    ax[0].barh(range(y_lower,y_upper),cluster_silhouette_vals,height =1)
    
    ax[0].text(-0.03,(y_lower+y_upper)/2,str(i+1))
    y_lower += len(cluster_silhouette_vals)

    avg_score = np.mean(silhoute_vals)
    ax[0].axvline(avg_score,linestyle ='--',linewidth =2,color = 'green')
    ax[0].set_yticks([])
    ax[0].set_xlim([-0.1, 1])
    ax[0].set_xlabel('Silhouette coefficient values')
    ax[0].set_ylabel('Cluster labels')
    ax[0].set_title('Silhouette plot for the various clusters');


    ax[1].scatter(X['Annual Income (k$)'],X['Spending Score (1-100)'] , c = labels2,cmap='viridis');
    ax[1].scatter(centroids2[:,0],centroids2[:,1],marker = '^' , c= 'r',s =250);
    ax[1].set_xlabel('Annual Income')
    ax[1].set_ylabel('Spending Score')
    ax[1].set_title('Visualization of clustered data', y=1.02)

    plt.tight_layout()
    plt.suptitle(f' Silhouette analysis using k = {k}',fontsize=16,fontweight = 'semibold')


In [ ]:
plt.scatter(np.arange(2,7),silh_score)

silhouette score shows the similarity score of point with others in its own cluster.

even though silhouette score is high for k = 5 , the silhouette plot for k = 5 showing neagative values for few points which shows average inter cluster distance of particular points greater than distance within the cluster. This might be due to noisy data that got clustered which are considered as outliers for particualr clusters 

Incase of noisy data. Its better to use DBSCAN algorithm for clustering with which we can seperate out noisy data.


## **With DBScan**

Two parameters to be defined for DBSCAN

*   Minpts: Mininum points that a given point should have to be considered as core point 
*   Epsilon: with point at center of circle neighbor hood is considered when the other point lie inside the radius of epsilon units. Epsilon is calculated by maximum optimal  distance between nneighbours to the points.

In [ ]:
min_pts =6 # choosing minpts depends on the domain or can be choosen by looking at the spread of data

#calculating epsilon
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=min_pts,metric='euclidean').fit(X[['Annual Income (k$)','Spending Score (1-100)']])
distances,indices = nbrs.kneighbors(X[['Annual Income (k$)','Spending Score (1-100)']])
distances = np.sort(distances,axis=0)
distances = distances[:,5]
plt.plot(distances)
plt.show()

In [ ]:
epsilon = 0.09 # from the above plot there is increase after elbow point 0.09

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=epsilon,min_samples=min_pts).fit(X[['Annual Income (k$)','Spending Score (1-100)']])
labels3 = dbscan.labels_

In [ ]:
pd.Series(labels3).value_counts()

In [ ]:
plt.scatter(X['Annual Income (k$)'],X['Spending Score (1-100)'],c=labels3,cmap='viridis')

In [ ]:
colors_map = {-1:'black',0:'green',1:'blue',2:'red',3:'yellow',4:'brown',5:'purple',6:'cyan'}
colorlist = ['green','blue','red','yellow','brown','purple','cyan']

In [ ]:
X['dbscan_clusters'] = labels3
Y = X[X['dbscan_clusters']!=-1]

In [ ]:
for each in range(0,pd.Series(labels3).nunique()-1):
  fig,ax = plt.subplots(1,1,figsize=(7,4))
  ax.scatter(X[(X['dbscan_clusters']==-1)|(X['dbscan_clusters']==each)]['Annual Income (k$)'],
              X[(X['dbscan_clusters']==-1)|(X['dbscan_clusters']==each)]['Spending Score (1-100)'],
              c=X[(X['dbscan_clusters']==-1)|(X['dbscan_clusters']==each)]['dbscan_clusters'].map(lambda a:colors_map[a]))
  plt.title('distribution of {} cluster and noise around'.format(each))

## validating clusters with Silhouette Score

In [ ]:
silh_score = []
#for k in range(2,7):
fig,ax = plt.subplots(1,1,figsize=(15,5))

silhoute_vals = silhouette_samples(Y[['Annual Income (k$)','Spending Score (1-100)']],Y['dbscan_clusters'])
#silh_score.append(silhouette_score(Y[['Annual Income (k$)','Spending Score (1-100)']],Y['dbscan_clusters'],metric='euclidean'))

y_ticks = []
y_lower = y_upper = 0

for i,cluster in enumerate(np.unique(Y['dbscan_clusters'])):
  cluster_silhouette_vals = silhoute_vals[Y['dbscan_clusters'] == cluster]
  cluster_silhouette_vals.sort()
  y_upper +=len(cluster_silhouette_vals)

  ax.barh(range(y_lower,y_upper),cluster_silhouette_vals,height =1)
    
  ax.text(-0.03,(y_lower+y_upper)/2,str(i))
  y_lower += len(cluster_silhouette_vals)

  avg_score = np.mean(silhoute_vals)
  ax.axvline(avg_score,linestyle ='--',linewidth =2,color = 'green')
  ax.set_yticks([])
  ax.set_xlim([-0.1, 1])
  ax.set_xlabel('Silhouette coefficient values')
  ax.set_ylabel('Cluster labels')
  ax.set_title('Silhouette plot for the various clusters');

  plt.tight_layout()

In [ ]:
silhouette_score(Y[['Annual Income (k$)','Spending Score (1-100)']],Y['dbscan_clusters'],metric='euclidean')

From the above silhouette score mapping , it is clear that average distance of few points from cluster 2 is closest to cluster 1 than cluster 2.This might be due to difference in cluster size of 1 and 2

DBSCAN is best when the clusters are in arbitary shape and since the name itself is density based it works well when the clusters are cllosely paked and of any shape.

Silhouette score of 0.5 is significant enough to consider clusters formed and it is similar to score obtained in the K-Means cluster


In [ ]:
X.head()

## **ClusterAnalysis**

In [ ]:
df1['Kmean_cluster'] = X['Clusters']

In [ ]:
df1.head()

In [ ]:
clstrAnalysis = df1.groupby(['Kmean_cluster'])['Annual Income (k$)','Spending Score (1-100)'].agg({'Annual Income (k$)':['min',np.mean,'max'],'Spending Score (1-100)':['min',np.mean,'max']})

In [ ]:
clstrAnalysis

In [ ]:
z = pd.DataFrame(np.hstack((np.array(clstrAnalysis['Annual Income (k$)']['mean'].rank()).reshape((-1,1)),
                            np.array(clstrAnalysis['Spending Score (1-100)']['mean'].rank()).reshape((-1,1)))),
                 columns=['RankA.Income','RankSpendScore'])

In [ ]:
clstrAnalysis2 = pd.concat([clstrAnalysis,z],axis=1)

In [ ]:
clstrAnalysis2

In [ ]:
plt.scatter(clstrAnalysis2['RankA.Income'],clstrAnalysis2['RankSpendScore'])
plt.xlabel('RankA.Income')
plt.ylabel('RankSpendScore')

In [ ]:
width=0.25
plt.bar(clstrAnalysis2.index,clstrAnalysis2.iloc[:,1],width=width,label='mean Annual Income')
plt.bar(clstrAnalysis2.index+width,clstrAnalysis2.iloc[:,4],width=width,label='mean Spending Score')
plt.ylabel('AnnualIncome-SpendingScore')
plt.xlabel('Cluster')
plt.legend()
plt.show()

From the above plot we can define the clusters as following:

*   Cluster3:Customers with Less income and High spending score,Less Income with mean of 25k$ and high spending score of mean about 79

*   Cluster0: Customers with Average Income and Average Spending Score when compared to other customers, Mean of Inome is around 55k$ and mean Spending Score of 49

*   Cluster1 :Customers with high income and very less expenditure, Mean of income is around 88k and mean spending score is around 17

*   Cluster4:Customers with high income and high expenditure, mean of income is around 86k$ and mean spending score around 82

*   Cluster2:Customers with less income and less expenditure, mean of income is around 26k$ and mean spending score around 20

## **Including Age**

since age has shown some pattern along with Spending , lets include Age to check whether we can form some interpretable clusters

In [ ]:
X = df2[['Age','Annual Income (k$)','Spending Score (1-100)']]

In [ ]:
X.head()

In [ ]:
inertia = []
for n in range(2,11):
  clstr = KMeans(n_clusters=n,init='k-means++',random_state=42)
  clstr.fit(X)
  inertia.append(clstr.inertia_)

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(np.arange(2,11),inertia,'-')
plt.plot(np.arange(2,11),inertia,'o',c='r')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Plot')
plt.show()

In [ ]:
clstr1 = KMeans(n_clusters=4,init='k-means++')
clstr1.fit(X)
labels1 = clstr1.labels_
centroids1 = clstr1.cluster_centers_
X['Clusters'] = labels1

In [ ]:
import plotly.express as px
fig = px.scatter_3d(X,x='Age',y='Annual Income (k$)',z='Spending Score (1-100)',color=pd.Series(labels1).map(lambda a:colors_map[a]))
fig.show()

## Validating Clusters with Silhouette Score

In [ ]:
silh_score = []
for k in range(2,8):
  fig,ax = plt.subplots(1,1,figsize=(15,5))
  km = KMeans(n_clusters=k,init='k-means++')
  clstr2 = km.fit(X[['Age','Annual Income (k$)','Spending Score (1-100)']])
  labels2 = clstr2.labels_
  centroids2 = clstr2.cluster_centers_
  silhoute_vals = silhouette_samples(X[['Age','Annual Income (k$)','Spending Score (1-100)']],labels2)
  silh_score.append(silhouette_score(X[['Age','Annual Income (k$)','Spending Score (1-100)']],labels2,metric='euclidean'))

  y_ticks = []
  y_lower = y_upper = 0
  for i,cluster in enumerate(np.unique(labels2)):
    cluster_silhouette_vals = silhoute_vals[labels2 == cluster]
    cluster_silhouette_vals.sort()
    y_upper +=len(cluster_silhouette_vals)

    ax.barh(range(y_lower,y_upper),cluster_silhouette_vals,height =1)
    
    ax.text(-0.03,(y_lower+y_upper)/2,str(i+1))
    y_lower += len(cluster_silhouette_vals)

    avg_score = np.mean(silhoute_vals)
    ax.axvline(avg_score,linestyle ='--',linewidth =2,color = 'green')
    ax.set_yticks([])
    ax.set_xlim([-0.1, 1])
    ax.set_xlabel('Silhouette coefficient values')
    ax.set_ylabel('Cluster labels')


    plt.tight_layout()
    plt.suptitle(f' Silhouette analysis using k = {k}',fontsize=16,fontweight = 'semibold')


clusters 2,3,4 having all silhouette scores in positive 5,6 clusters might have some points wrongly clustered which can be seen in respective clusters in the above plot with negative values

In [ ]:
plt.scatter(np.arange(2,8),silh_score)
plt.xlabel('cluster')
plt.ylabel('silhouette_score')

We can stick with number of clusters as 4 even though silhouette score is less when compared to 5 and 6 but the silhouette values are all positive which is not the case with 5 and 6 clusters.

In [ ]:
df1['cluster_labels2'] = labels1

In [ ]:
df1['cluster_labels2'].value_counts()

## **Cluster Analysis**

In [ ]:
df1.groupby(['cluster_labels2'])['Age','Annual Income (k$)','Spending Score (1-100)'].agg({'Age':['min',np.mean,'max'],'Annual Income (k$)':['min',np.mean,'max'],'Spending Score (1-100)':['min',np.mean,'max']})

In [ ]:
clstrAnalysis = df1.groupby(['cluster_labels2'])['Age','Annual Income (k$)','Spending Score (1-100)'].agg({'Age':['min',np.mean,'max'],'Annual Income (k$)':['min',np.mean,'max'],'Spending Score (1-100)':['min',np.mean,'max']})

clusters are ranked according to mean values of Age,Income,SpendingScore where 1 being the least mean 

In [ ]:
z = pd.DataFrame(np.hstack((np.array(clstrAnalysis['Age']['mean'].rank()).reshape((-1,1)),
                            np.array(clstrAnalysis['Annual Income (k$)']['mean'].rank()).reshape((-1,1)),
                            np.array(clstrAnalysis['Spending Score (1-100)']['mean'].rank()).reshape((-1,1)))),
                 columns=['RankAge','RankA.Income','RankSpendScore'])

In [ ]:
clstrAnalysis2 = pd.concat([clstrAnalysis,z],axis=1)

In [ ]:
clstrAnalysis2

In [ ]:
fig = px.scatter_3d(clstrAnalysis2,x='RankAge',y='RankA.Income',z='RankSpendScore')
fig.show()

In [ ]:
width=0.25
plt.bar(clstrAnalysis2.index-width,clstrAnalysis2.iloc[:,1],width=width,label='AgeRank')
plt.bar(clstrAnalysis2.index,clstrAnalysis2.iloc[:,4],width=width,label='RankIncome')
plt.bar(clstrAnalysis2.index+width,clstrAnalysis2.iloc[:,7],width=width,label='RankSpendScore')
plt.ylabel('Age-AnnualIncome-SpendingScore')
plt.xlabel('Cluster')
plt.legend()
plt.show()

From the above plots we can interpret the clusters as following

*     Cluster0:Customers with minimum age group(mean 25) ,less income(mean 40) and medium spending score(mean 60).
*     Cluster1:Customers with medium Age(mean 32), high Income(mean 86) and High Sending Score(mean 81)
*     Cluster2:Customers with High Age(mean 53), medium income(mean 47) and Average Spending Score(mean 39).
*     Cluster3:Customers with medium Age(mean 39),High Income(mean86) and Minimum Spending Score(mean 19).

every cluster is unique.
